In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# from wordcloud import WordCloud
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay
# from sklearn.pipeline import Pipeline
# from scipy.stats import uniform
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
# from datasets import Dataset
# import torch

# # Load the data

# # Load the data
# try:
#     train_df = pd.read_csv('/kaggle/input/mercor-ai-detection/train.csv')
#     test_df = pd.read_csv('/kaggle/input/mercor-ai-detection/test.csv')
#     sample_submission_df = pd.read_csv('/kaggle/input/mercor-ai-detection/sample_submission.csv')
# except FileNotFoundError:
#     print("Ensure train.csv, test.csv, and sample_submission.csv are in the same directory.")
#     # Create dummy dataframes for demonstration if files are not found
#     train_df = pd.DataFrame({
#         'id': range(100),
#         'topic': ['topic_' + str(i % 5) for i in range(100)],
#         'answer': ['This is a sample answer ' * 10 for i in range(100)],
#         'is_cheating': np.random.randint(0, 2, 100)
#     })
#     test_df = pd.DataFrame({
#         'id': range(100),
#         'topic': ['topic_' + str(i % 5) for i in range(100)],
#         'answer': ['This is a sample test answer ' * 10 for i in range(100)]
#     })

# # --- 1. Exploratory Data Analysis (EDA) ---
# print("Performing Exploratory Data Analysis...")

# # Plot 1: Distribution of Target Variable
# plt.figure(figsize=(6, 6))
# train_df['is_cheating'].value_counts().plot.pie(autopct='%1.1f%%', startangle=90, colors=['skyblue', 'salmon'])
# plt.title('Distribution of Target Variable (is_cheating)')
# plt.ylabel('')
# plt.show()

# # Plot 2: Text Length Analysis
# train_df['answer_length'] = train_df['answer'].str.len()
# plt.figure(figsize=(10, 6))
# sns.histplot(data=train_df, x='answer_length', hue='is_cheating', kde=True, bins=50)
# plt.title('Distribution of Answer Length by Class')
# plt.xlabel('Answer Length')
# plt.ylabel('Frequency')
# plt.show()

# # Plot 3: Topic Distribution
# plt.figure(figsize=(12, 6))
# sns.countplot(y='topic', data=train_df, order=train_df['topic'].value_counts().index, palette='viridis')
# plt.title('Distribution of Topics')
# plt.xlabel('Count')
# plt.ylabel('Topic')
# plt.show()

# # Plot 4 & 5: Word Clouds
# cheating_text = ' '.join(train_df[train_df['is_cheating'] == 1]['answer'].dropna())
# non_cheating_text = ' '.join(train_df[train_df['is_cheating'] == 0]['answer'].dropna())

# wordcloud_cheating = WordCloud(width=800, height=400, background_color='white').generate(cheating_text)
# wordcloud_non_cheating = WordCloud(width=800, height=400, background_color='white').generate(non_cheating_text)

# fig, axes = plt.subplots(1, 2, figsize=(20, 10))
# axes[0].imshow(wordcloud_cheating, interpolation='bilinear')
# axes[0].set_title('Word Cloud for Cheating Class')
# axes[0].axis('off')
# axes[1].imshow(wordcloud_non_cheating, interpolation='bilinear')
# axes[1].set_title('Word Cloud for Non-Cheating Class')
# axes[1].axis('off')
# plt.show()

# # Data Preprocessing for Models
# X = train_df['answer']
# y = train_df['is_cheating']
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # --- 2. Modeling ---
# print("Training and evaluating models...")
# results = {}

# # --- Model 1: Logistic Regression with TF-IDF and RandomizedSearchCV ---
# print("Training Logistic Regression with TF-IDF...")
# pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer()),
#     ('clf', LogisticRegression(solver='liblinear', random_state=42))
# ])

# param_dist = {
#     'tfidf__ngram_range': [(1, 1), (1, 2)],
#     'tfidf__max_df': uniform(0.5, 0.5),
#     'tfidf__min_df': range(1, 11),
#     'clf__C': uniform(0.1, 10),
#     'clf__penalty': ['l1', 'l2']
# }

# random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=10, cv=5,
#                                    scoring='roc_auc', random_state=42, n_jobs=-1)
# random_search.fit(X_train, y_train)

# best_lr = random_search.best_estimator_
# y_pred_lr = best_lr.predict_proba(X_val)[:, 1]
# roc_auc_lr = roc_auc_score(y_val, y_pred_lr)
# results['Logistic Regression'] = {'roc_auc': roc_auc_lr, 'y_pred': y_pred_lr}
# print(f"Logistic Regression ROC-AUC: {roc_auc_lr:.4f}")

# # Plot 6: Logistic Regression ROC Curve and Confusion Matrix
# fpr_lr, tpr_lr, _ = roc_curve(y_val, y_pred_lr)
# cm_lr = confusion_matrix(y_val, (y_pred_lr > 0.5).astype(int))

# fig, axes = plt.subplots(1, 2, figsize=(14, 6))
# axes[0].plot(fpr_lr, tpr_lr, label=f'ROC curve (area = {roc_auc_lr:.2f})')
# axes[0].plot([0, 1], [0, 1], 'k--')
# axes[0].set_xlim([0.0, 1.0])
# axes[0].set_ylim([0.0, 1.05])
# axes[0].set_xlabel('False Positive Rate')
# axes[0].set_ylabel('True Positive Rate')
# axes[0].set_title('Logistic Regression ROC Curve')
# axes[0].legend(loc="lower right")

# ConfusionMatrixDisplay(confusion_matrix=cm_lr).plot(ax=axes[1], cmap='Blues')
# axes[1].set_title('Logistic Regression Confusion Matrix')
# plt.show()

# # --- Transformer Models ---
# def train_transformer(model_name, train_texts, train_labels, val_texts, val_labels):
#     train_df_hf = pd.DataFrame({'text': train_texts.tolist(), 'label': train_labels.tolist()})
#     val_df_hf = pd.DataFrame({'text': val_texts.tolist(), 'label': val_labels.tolist()})

#     train_dataset = Dataset.from_pandas(train_df_hf)
#     val_dataset = Dataset.from_pandas(val_df_hf)

#     tokenizer = AutoTokenizer.from_pretrained(model_name)

#     def tokenize_function(examples):
#         return tokenizer(examples['text'], padding="max_length", truncation=True)

#     tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
#     tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

#     model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

#     training_args = TrainingArguments(
#         output_dir=f'./results_{model_name.replace("/", "_")}',
#         evaluation_strategy="epoch",
#         learning_rate=2e-5,
#         per_device_train_batch_size=8,
#         per_device_eval_batch_size=8,
#         num_train_epochs=1, # Using 1 epoch for demonstration
#         weight_decay=0.01,
#         logging_dir='./logs',
#     )

#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=tokenized_train_dataset,
#         eval_dataset=tokenized_val_dataset,
#     )

#     trainer.train()

#     predictions = trainer.predict(tokenized_val_dataset)
#     probs = torch.softmax(torch.from_numpy(predictions.predictions), dim=-1)[:, 1].numpy()
#     return probs

# transformer_models = {
#     'BERT': 'bert-base-uncased',
#     'RoBERTa': 'roberta-base',
#     'DistilBERT': 'distilbert-base-uncased'
# }

# for model_name, model_path in transformer_models.items():
#     print(f"Training {model_name}...")
#     try:
#         y_pred_transformer = train_transformer(model_path, X_train, y_train, X_val, y_val)
#         roc_auc_transformer = roc_auc_score(y_val, y_pred_transformer)
#         results[model_name] = {'roc_auc': roc_auc_transformer, 'y_pred': y_pred_transformer}
#         print(f"{model_name} ROC-AUC: {roc_auc_transformer:.4f}")

#         # Plot ROC Curve and Confusion Matrix for each transformer model
#         fpr_transformer, tpr_transformer, _ = roc_curve(y_val, y_pred_transformer)
#         cm_transformer = confusion_matrix(y_val, (y_pred_transformer > 0.5).astype(int))

#         fig, axes = plt.subplots(1, 2, figsize=(14, 6))
#         axes[0].plot(fpr_transformer, tpr_transformer, label=f'ROC curve (area = {roc_auc_transformer:.2f})')
#         axes[0].plot([0, 1], [0, 1], 'k--')
#         axes[0].set_xlim([0.0, 1.0])
#         axes[0].set_ylim([0.0, 1.05])
#         axes[0].set_xlabel('False Positive Rate')
#         axes[0].set_ylabel('True Positive Rate')
#         axes[0].set_title(f'{model_name} ROC Curve')
#         axes[0].legend(loc="lower right")

#         ConfusionMatrixDisplay(confusion_matrix=cm_transformer).plot(ax=axes[1], cmap='Blues')
#         axes[1].set_title(f'{model_name} Confusion Matrix')
#         plt.show()

#     except Exception as e:
#         print(f"Error training {model_name}: {e}")
#         print(f"Skipping {model_name} due to the error.")

# # --- 3. Model Comparison ---
# print("Comparing model performance...")

# # Plot 7: Comparison of ROC-AUC Scores
# model_names = list(results.keys())
# roc_auc_scores = [res['roc_auc'] for res in results.values()]

# plt.figure(figsize=(10, 6))
# sns.barplot(x=model_names, y=roc_auc_scores, palette='mako')
# plt.title('Comparison of Model ROC-AUC Scores')
# plt.xlabel('Model')
# plt.ylabel('ROC-AUC Score')
# plt.ylim(0.5, 1.0)
# for i, score in enumerate(roc_auc_scores):
#     plt.text(i, score + 0.01, f'{score:.4f}', ha='center')
# plt.show()

# # Plot 8: Combined ROC Curves
# plt.figure(figsize=(10, 8))
# for model_name, result in results.items():
#     fpr, tpr, _ = roc_curve(y_val, result['y_pred'])
#     plt.plot(fpr, tpr, label=f"{model_name} (AUC = {result['roc_auc']:.4f})")
# plt.plot([0, 1], [0, 1], 'k--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Combined ROC Curves of All Models')
# plt.legend(loc="lower right")
# plt.show()


# # --- 4. Submission ---
# print("Generating submission file...")
# if results:
#     best_model_name = max(results, key=lambda k: results[k]['roc_auc'])
#     print(f"Best performing model: {best_model_name}")

#     if 'Logistic Regression' in best_model_name:
#         test_pred_probs = best_lr.predict_proba(test_df['answer'])[:, 1]
#     else:
#         # For a real submission, the best transformer model should be retrained on the full training data.
#         # Here, we'll just use the validation model for prediction as a demonstration.
#         test_texts = test_df['answer'].tolist()
#         # This part requires re-running the prediction on the test set with the best transformer model.
#         # For simplicity, we are creating a dummy prediction here.
#         test_pred_probs = np.random.rand(len(test_df))
#         print("Note: For a real submission with a transformer, retrain on full data.")


#     submission_df = pd.DataFrame({'id': test_df['id'], 'is_cheating': test_pred_probs})
#     # --- 4. Submission ---
#     submission_df.to_csv('/kaggle/working/submission.csv', index=False)
#     print("Submission file '/kaggle/working/submission.csv' created successfully.")

#     print("Submission file 'submission.csv' created successfully.")

#     # Plot 9: Distribution of Predicted Probabilities on Test Set
#     plt.figure(figsize=(10, 6))
#     sns.histplot(test_pred_probs, bins=50, kde=True)
#     plt.title(f'Distribution of Predicted Probabilities on Test Set ({best_model_name})')
#     plt.xlabel('Predicted Probability')
#     plt.ylabel('Frequency')
#     plt.show()
# else:
#     print("No models were successfully trained. Submission file not generated.")

In [ ]:
# run in a cell
!python --version
import sys, subprocess
print("sys.version:", sys.version)
!pip show torch || true
!pip show torch_xla || true
!pip install cloud-tpu-client==0.10
!pip install torch==2.0.0 torchvision==0.15.1 \
  https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl

In [ ]:
# # ===================================================================
# # 1. SETUP AND INITIAL CONFIGURATION
# # ===================================================================
# import os
# import random
# import time
# import warnings
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import torch
# import torch.nn as nn
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import (roc_auc_score, confusion_matrix,
#                              ConfusionMatrixDisplay, roc_curve, auc,
#                              precision_recall_curve)
# from transformers import (AutoTokenizer, AutoConfig,
#                             AutoModelForSequenceClassification,
#                             DataCollatorWithPadding, TrainingArguments,
#                             Trainer, EarlyStoppingCallback,
#                             logging as transformers_logging)
# from datasets import Dataset

# # --- Core Settings ---
# # Input and output paths for Kaggle environment
# DATA_DIR = "/kaggle/input/mercor-ai-detection"
# OUTPUT_SUBMISSION_PATH = "/kaggle/working/submission.csv"
# PLOT_DIR = "/kaggle/working/plots"
# MODEL_OUTPUT_DIR = "/kaggle/working/finetuned_model"

# # Create directories for outputs
# os.makedirs(PLOT_DIR, exist_ok=True)
# os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)

# # --- Model & Training Hyperparameters ---
# # Using a model pre-trained specifically for AI text detection
# MODEL_NAME = "fakespot-ai/roberta-base-ai-text-detection-v1"
# MAX_TOKEN_LENGTH = 512
# TRAIN_BATCH_SIZE = 16  # Adjusted for better stability on various GPUs
# EVAL_BATCH_SIZE = 16
# NUM_EPOCHS = 8
# LEARNING_RATE = 2e-5
# SEED = 42

# # --- Reproducibility ---
# def set_seed(seed_value):
#     """Set seed for reproducibility."""
#     random.seed(seed_value)
#     np.random.seed(seed_value)
#     torch.manual_seed(seed_value)
#     if torch.cuda.is_available():
#         torch.cuda.manual_seed_all(seed_value)

# set_seed(SEED)
# warnings.filterwarnings("ignore", category=UserWarning)
# transformers_logging.set_verbosity_error()

# print("Setup Complete. Using model:", MODEL_NAME)

# # ===================================================================
# # 2. DATA LOADING AND PREPARATION
# # ===================================================================
# # --- Helper Functions for Text Cleaning ---
# def clean_text(text):
#     """Removes extra whitespace, newlines, and returns empty string for NaNs."""
#     if pd.isna(text):
#         return ""
#     return " ".join(str(text).strip().replace("\r", " ").replace("\n", " ").split())

# def format_input_text(topic, answer):
#     """Creates a structured input string from topic and answer."""
#     clean_topic = clean_text(topic)
#     clean_answer = clean_text(answer)
#     return f"TOPIC: {clean_topic}\n\nANSWER: {clean_answer}"

# # --- Load DataFrames ---
# train_full_df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
# test_df = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))

# # --- Preprocess and Create Input Text ---
# train_full_df["text"] = train_full_df.apply(lambda row: format_input_text(row["topic"], row["answer"]), axis=1)
# train_full_df["label"] = train_full_df["is_cheating"].astype(int)

# test_df["text"] = test_df.apply(lambda row: format_input_text(row.get("topic", ""), row.get("answer", "")), axis=1)
# print(f"Data loaded. Train samples: {len(train_full_df)}, Test samples: {len(test_df)}")

# # --- Data Augmentation: Upsampling Minority Class ---
# label_counts = train_full_df["label"].value_counts()
# print("Original label distribution:\n", label_counts)

# minority_label = label_counts.idxmin()
# majority_count = label_counts.max()
# minority_count = label_counts.min()

# if minority_count < majority_count:
#     samples_to_add = majority_count - minority_count
#     minority_df = train_full_df[train_full_df["label"] == minority_label]
#     upsampled_minority = minority_df.sample(n=samples_to_add, replace=True, random_state=SEED)
    
#     train_balanced_df = pd.concat([train_full_df, upsampled_minority]).sample(frac=1, random_state=SEED).reset_index(drop=True)
#     print(f"\nUpsampled label '{minority_label}' by adding {samples_to_add} samples.")
#     print("New balanced label distribution:\n", train_balanced_df['label'].value_counts())
# else:
#     train_balanced_df = train_full_df.copy()
#     print("\nNo upsampling needed as classes are balanced.")

# # --- Train/Validation Split (Stratified) ---
# train_df, val_df = train_test_split(
#     train_balanced_df[["text", "label"]],
#     test_size=0.20,
#     random_state=SEED,
#     stratify=train_balanced_df["label"]
# )
# print(f"\nData split. Train size: {len(train_df)}, Validation size: {len(val_df)}")

# # --- Convert to Hugging Face Dataset objects ---
# train_dataset = Dataset.from_pandas(train_df)
# val_dataset = Dataset.from_pandas(val_df)
# test_dataset = Dataset.from_pandas(test_df[["id", "text"]].rename(columns={"id": "original_id"}))

# # ===================================================================
# # 3. MODEL CUSTOMIZATION AND TOKENIZATION
# # ===================================================================
# # --- Tokenizer ---
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# # --- Custom Model Head (ANN Classifier) ---
# class CustomANNHead(nn.Module):
#     """A custom classifier head with multiple layers for improved performance."""
#     def __init__(self, input_size, inner_dim=512, dropout_rate=0.2):
#         super().__init__()
#         self.fc1 = nn.Linear(input_size, inner_dim)
#         self.activation1 = nn.ReLU()
#         self.dropout1 = nn.Dropout(dropout_rate)
#         self.fc2 = nn.Linear(inner_dim, inner_dim // 2)
#         self.activation2 = nn.ReLU()
#         self.dropout2 = nn.Dropout(dropout_rate)
#         self.output_layer = nn.Linear(inner_dim // 2, 2) # 2 for binary classification

#         # Initialize weights for better stability
#         nn.init.xavier_uniform_(self.fc1.weight)
#         nn.init.xavier_uniform_(self.fc2.weight)
#         nn.init.xavier_uniform_(self.output_layer.weight)

#     def forward(self, features):
#         # The RoBERTa model outputs a sequence of hidden states.
#         # We use the representation of the first token ([CLS]) for classification.
#         cls_token_features = features[:, 0, :]
        
#         x = self.dropout1(self.activation1(self.fc1(cls_token_features)))
#         x = self.dropout2(self.activation2(self.fc2(x)))
#         logits = self.output_layer(x)
#         return logits

# # --- Load Base Model and Attach Custom Head ---
# model_config = AutoConfig.from_pretrained(MODEL_NAME, num_labels=2)
# base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=model_config)

# # Replace the original classifier with our custom, more powerful one
# hidden_layer_size = base_model.config.hidden_size
# base_model.classifier = CustomANNHead(hidden_layer_size)
# base_model.config.use_cache = False # Important for training
# print("\nCustom ANN head attached to the model.")

# # --- Tokenize Datasets ---
# def tokenize_data(batch):
#     return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_TOKEN_LENGTH)

# print("Tokenizing datasets...")
# train_dataset = train_dataset.map(tokenize_data, batched=True).rename_column("label", "labels")
# val_dataset = val_dataset.map(tokenize_data, batched=True).rename_column("label", "labels")
# test_dataset = test_dataset.map(tokenize_data, batched=True)

# train_dataset.set_format("torch")
# val_dataset.set_format("torch")
# test_dataset.set_format("torch", columns=['input_ids', 'attention_mask', 'original_id'])

# # ===================================================================
# # 4. TRAINING THE MODEL
# # ===================================================================
# # --- Metrics Calculation ---
# def compute_metrics(eval_predictions):
#     logits, labels = eval_predictions
#     probabilities = torch.softmax(torch.from_numpy(logits), dim=1).numpy()[:, 1]
#     roc_auc = roc_auc_score(labels, probabilities)
#     predictions = np.argmax(logits, axis=1)
#     accuracy = (predictions == labels).mean()
#     return {"roc_auc": roc_auc, "accuracy": accuracy}

# # --- Training Arguments ---
# training_args = TrainingArguments(
#     output_dir=MODEL_OUTPUT_DIR,
#     # Use 'eval_strategy' which is the correct argument name
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     per_device_train_batch_size=TRAIN_BATCH_SIZE,
#     per_device_eval_batch_size=EVAL_BATCH_SIZE,
#     num_train_epochs=NUM_EPOCHS,
#     learning_rate=LEARNING_RATE,
#     weight_decay=0.01,
#     fp16=torch.cuda.is_available(), # Enable mixed-precision training if GPU is available
#     logging_steps=20,
#     save_total_limit=2,
#     load_best_model_at_end=True,
#     metric_for_best_model="roc_auc",
#     greater_is_better=True,
#     report_to="none",
# )

# # --- Initialize Trainer ---
# trainer = Trainer(
#     model=base_model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     data_collator=DataCollatorWithPadding(tokenizer),
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
# )

# # --- Start Training ---
# print("\nStarting model training...")
# start_time = time.time()
# trainer.train()
# end_time = time.time()
# print(f"Training finished in {(end_time - start_time) / 60:.2f} minutes.")

# # ===================================================================
# # 5. EVALUATION AND VISUALIZATION
# # ===================================================================
# print("\nEvaluating model and generating visualizations...")

# # --- Get Validation Predictions ---
# val_predictions = trainer.predict(val_dataset)
# val_logits = val_predictions.predictions
# val_probs = torch.softmax(torch.from_numpy(val_logits), dim=1).numpy()[:, 1]
# val_true_labels = val_predictions.label_ids
# val_pred_labels = np.argmax(val_logits, axis=1)
# final_roc_auc = roc_auc_score(val_true_labels, val_probs)

# print(f"Final Validation ROC-AUC on the best model: {final_roc_auc:.4f}")

# # --- Plot 1 & 2: ROC and Precision-Recall Curves ---
# fpr, tpr, _ = roc_curve(val_true_labels, val_probs)
# precision, recall, _ = precision_recall_curve(val_true_labels, val_probs)

# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# ax1.plot(fpr, tpr, label=f'ROC Curve (AUC = {auc(fpr, tpr):.4f})', color='darkorange')
# ax1.plot([0, 1], [0, 1], 'k--', label='Chance')
# ax1.set_xlabel("False Positive Rate")
# ax1.set_ylabel("True Positive Rate")
# ax1.set_title("Validation ROC Curve")
# ax1.legend()
# ax1.grid(True, alpha=0.3)

# ax2.plot(recall, precision, label='Precision-Recall Curve', color='cornflowerblue')
# ax2.set_xlabel("Recall")
# ax2.set_ylabel("Precision")
# ax2.set_title("Validation Precision-Recall Curve")
# ax2.legend()
# ax2.grid(True, alpha=0.3)
# plt.tight_layout()
# plt.savefig(os.path.join(PLOT_DIR, "roc_pr_curves.png"))
# plt.show()

# # --- Plot 3 & 4: Confusion Matrix and Normalized Confusion Matrix ---
# cm = confusion_matrix(val_true_labels, val_pred_labels)
# cm_normalized = confusion_matrix(val_true_labels, val_pred_labels, normalize='true')

# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
# ConfusionMatrixDisplay(cm, display_labels=["Human", "AI/Cheating"]).plot(ax=ax1, cmap="Blues")
# ax1.set_title("Validation Confusion Matrix")
# ConfusionMatrixDisplay(cm_normalized, display_labels=["Human", "AI/Cheating"]).plot(ax=ax2, cmap="Greens")
# ax2.set_title("Normalized Confusion Matrix")
# plt.tight_layout()
# plt.savefig(os.path.join(PLOT_DIR, "confusion_matrices.png"))
# plt.show()

# # --- Plot 5 & 6: Training/Validation Loss and ROC-AUC over Epochs ---
# log_history = pd.DataFrame(trainer.state.log_history)
# eval_logs = log_history[log_history['eval_loss'].notna()].copy()
# train_logs = log_history[log_history['loss'].notna() & log_history['epoch'].notna()].copy()

# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
# if not train_logs.empty and not eval_logs.empty:
#     ax1.plot(train_logs['epoch'], train_logs['loss'], marker='o', linestyle='--', label='Training Loss')
#     ax1.plot(eval_logs['epoch'], eval_logs['eval_loss'], marker='o', label='Validation Loss')
#     ax1.set_xlabel("Epoch")
#     ax1.set_ylabel("Loss")
#     ax1.set_title("Training & Validation Loss")
#     ax1.legend()
#     ax1.grid(True, alpha=0.3)

#     ax2.plot(eval_logs['epoch'], eval_logs['eval_roc_auc'], marker='o', label='Validation ROC-AUC', color='teal')
#     ax2.set_xlabel("Epoch")
#     ax2.set_ylabel("ROC-AUC")
#     ax2.set_title("Validation ROC-AUC per Epoch")
#     ax2.legend()
#     ax2.grid(True, alpha=0.3)
# plt.tight_layout()
# plt.savefig(os.path.join(PLOT_DIR, "training_history.png"))
# plt.show()

# # ===================================================================
# # 6. SUBMISSION FILE GENERATION
# # ===================================================================
# print("\nPredicting on the test set for submission...")
# test_predictions = trainer.predict(test_dataset.remove_columns('original_id'))
# test_logits = test_predictions.predictions
# test_probs = torch.softmax(torch.from_numpy(test_logits), dim=1).numpy()[:, 1]

# # --- Create Submission DataFrame ---
# submission_df = pd.DataFrame({
#     "id": test_dataset["original_id"],
#     "is_cheating": test_probs
# })
# submission_df.to_csv(OUTPUT_SUBMISSION_PATH, index=False)
# print(f"Submission file created at: {OUTPUT_SUBMISSION_PATH}")
# print(submission_df.head())

# # --- Plot 7 & 8: Distribution of Test Predictions and Validation vs. Test ---
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
# sns.histplot(submission_df['is_cheating'], bins=50, kde=True, ax=ax1, color='purple')
# ax1.set_title("Distribution of Predicted Probabilities (Test Set)")
# ax1.set_xlabel("Predicted Probability of Cheating")

# sns.kdeplot(val_probs, ax=ax2, label='Validation Predictions', fill=True)
# sns.kdeplot(submission_df['is_cheating'], ax=ax2, label='Test Predictions', fill=True)
# ax2.set_title("Validation vs. Test Prediction Distributions")
# ax2.set_xlabel("Predicted Probability")
# ax2.legend()

# plt.tight_layout()
# plt.savefig(os.path.join(PLOT_DIR, "prediction_distributions.png"))
# plt.show()

# print("\nProcess complete. All plots saved in:", PLOT_DIR)

In [ ]:
# ===================================================================
# FINAL SUBMISSION SCRIPT: ENSEMBLE OF MULTIPLE MODELS
# ===================================================================
# This script will:
# 1. Train our primary model (Specialized RoBERTa) as before.
# 2. Train a second, powerful transformer (DeBERTa-v3) for diversity.
# 3. Blend the predictions of these models to create a more robust
#    and accurate final submission.
# ===================================================================

# ===================================================================
# 1. SETUP AND INITIAL CONFIGURATION
# ===================================================================
import os
import random
import time
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from transformers import (AutoTokenizer, AutoConfig,
                            AutoModelForSequenceClassification,
                            DataCollatorWithPadding, TrainingArguments,
                            Trainer, EarlyStoppingCallback,
                            logging as transformers_logging)
from datasets import Dataset
from scipy.stats import rankdata

# --- Core Settings ---
DATA_DIR = "/kaggle/input/mercor-ai-detection"
OUTPUT_DIR = "/kaggle/working/"
PLOT_DIR = os.path.join(OUTPUT_DIR, "plots")
os.makedirs(PLOT_DIR, exist_ok=True)

# --- Model & Training Hyperparameters ---
MODEL_1_NAME = "fakespot-ai/roberta-base-ai-text-detection-v1"
MODEL_2_NAME = "microsoft/deberta-v3-base"
MAX_TOKEN_LENGTH = 512
TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 16
NUM_EPOCHS = 10 # Reduced slightly for faster execution of two models
LEARNING_RATE = 2e-5
SEED = 42

# --- Reproducibility ---
def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)

set_seed(SEED)
warnings.filterwarnings("ignore", category=UserWarning)
transformers_logging.set_verbosity_error()

print("Setup Complete.")

# ===================================================================
# 2. DATA LOADING AND PREPARATION (COMMON FOR ALL MODELS)
# ===================================================================
# --- Helper Functions for Text Cleaning ---
def clean_text(text):
    if pd.isna(text): return ""
    return " ".join(str(text).strip().replace("\r", " ").replace("\n", " ").split())

def format_input_text(topic, answer):
    return f"TOPIC: {clean_text(topic)}\n\nANSWER: {clean_text(answer)}"

# --- Load DataFrames ---
train_full_df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
test_df = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))

# --- Preprocess and Create Input Text ---
train_full_df["text"] = train_full_df.apply(lambda row: format_input_text(row["topic"], row["answer"]), axis=1)
train_full_df["label"] = train_full_df["is_cheating"].astype(int)
test_df["text"] = test_df.apply(lambda row: format_input_text(row.get("topic", ""), row.get("answer", "")), axis=1)

# --- Upsampling Minority Class ---
label_counts = train_full_df["label"].value_counts()
minority_label = label_counts.idxmin()
if label_counts.min() < label_counts.max():
    samples_to_add = label_counts.max() - label_counts.min()
    minority_df = train_full_df[train_full_df["label"] == minority_label]
    upsampled_minority = minority_df.sample(n=samples_to_add, replace=True, random_state=SEED)
    train_balanced_df = pd.concat([train_full_df, upsampled_minority]).sample(frac=1, random_state=SEED).reset_index(drop=True)
else:
    train_balanced_df = train_full_df.copy()

# --- Train/Validation Split ---
train_df, val_df = train_test_split(
    train_balanced_df[["text", "label"]],
    test_size=0.20,
    random_state=SEED,
    stratify=train_balanced_df["label"]
)

# ===================================================================
# 3. GENERIC TRAINING AND PREDICTION FUNCTION
# ===================================================================
def train_and_predict(model_name, train_data, val_data, test_data):
    """A reusable function to train a transformer model and return test predictions."""
    print(f"\n--- Starting Training for: {model_name} ---")
    
    # Convert to HF Datasets
    train_ds = Dataset.from_pandas(train_data)
    val_ds = Dataset.from_pandas(val_data)
    test_ds = Dataset.from_pandas(test_data)

    # Load Tokenizer and Model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    model.config.use_cache = False

    # Tokenize
    def tokenize(batch):
        return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_TOKEN_LENGTH)
        
    train_ds = train_ds.map(tokenize, batched=True).rename_column("label", "labels")
    val_ds = val_ds.map(tokenize, batched=True).rename_column("label", "labels")
    test_ds = test_ds.map(tokenize, batched=True)

    train_ds.set_format("torch")
    val_ds.set_format("torch")
    test_ds.set_format("torch", columns=['input_ids', 'attention_mask'])

    def compute_metrics(p):
        logits, labels = p
        probs = torch.softmax(torch.from_numpy(logits), dim=1).numpy()[:, 1]
        return {"roc_auc": roc_auc_score(labels, probs)}

    training_args = TrainingArguments(
        output_dir=os.path.join(OUTPUT_DIR, f"model_{model_name.replace('/', '_')}"),
        eval_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=8 if "deberta" in model_name.lower() else TRAIN_BATCH_SIZE,
        gradient_accumulation_steps=2 if "deberta" in model_name.lower() else 1,

        per_device_eval_batch_size=EVAL_BATCH_SIZE,
        num_train_epochs=NUM_EPOCHS,
        learning_rate=LEARNING_RATE,
        weight_decay=0.01,
        # fp16=torch.cuda.is_available(),
        bf16=True if torch.cuda.is_available() else False,  # more memory-efficient precision

        logging_steps=50,
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="roc_auc",
        greater_is_better=True,
        report_to="none",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=DataCollatorWithPadding(tokenizer),
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    trainer.train()

    # Get predictions on the test set
    test_preds = trainer.predict(test_ds)
    test_probs = torch.softmax(torch.from_numpy(test_preds.predictions), dim=1).numpy()[:, 1]
    
    # Get validation score for reference
    val_preds = trainer.predict(val_ds)
    val_probs = torch.softmax(torch.from_numpy(val_preds.predictions), dim=1).numpy()[:, 1]
    # NEW CODE (corrected)
    val_score = roc_auc_score(val_ds['labels'], val_probs)
    print(f"--- Finished {model_name}. Validation ROC-AUC: {val_score:.4f} ---")
    
    return test_probs, val_score

# ===================================================================
# 4. TRAIN MODELS AND COLLECT PREDICTIONS
# ===================================================================
predictions = {}
model_scores = {}

# Train and predict with Model 1
preds_1, score_1 = train_and_predict(MODEL_1_NAME, train_df, val_df, test_df)
predictions['model_1'] = preds_1
model_scores['model_1'] = score_1

# Train and predict with Model 2
preds_2, score_2 = train_and_predict(MODEL_2_NAME, train_df, val_df, test_df)
predictions['model_2'] = preds_2
model_scores['model_2'] = score_2

# ===================================================================
# 5. ENSEMBLE PREDICTIONS
# ===================================================================
print("\n--- Ensembling Predictions ---")

# Create a DataFrame with all predictions
pred_df = pd.DataFrame(predictions)
pred_df['id'] = test_df['id']

# --- Strategy 1: Weighted Averaging based on Validation Score ---
# Give more weight to the model that performed better on the validation set.
total_score = sum(model_scores.values())
weight_1 = model_scores['model_1'] / total_score
weight_2 = model_scores['model_2'] / total_score

pred_df['weighted_avg'] = (pred_df['model_1'] * weight_1) + \
                          (pred_df['model_2'] * weight_2)

print(f"Model 1 Val Score: {score_1:.4f}, Assigned Weight: {weight_1:.4f}")
print(f"Model 2 Val Score: {score_2:.4f}, Assigned Weight: {weight_2:.4f}")

# --- Strategy 2: Rank Averaging ---
# This method is robust to outliers where one model is overly confident.
pred_df['rank_1'] = rankdata(pred_df['model_1'])
pred_df['rank_2'] = rankdata(pred_df['model_2'])
pred_df['avg_rank'] = (pred_df['rank_1'] + pred_df['rank_2']) / 2
# Scale the ranks back to a [0, 1] range
pred_df['rank_ensembled'] = (pred_df['avg_rank'] - pred_df['avg_rank'].min()) / \
                             (pred_df['avg_rank'].max() - pred_df['avg_rank'].min())

# --- Final Blended Prediction ---
# We will create a final blend of the weighted average and the rank average.
# This often produces the most stable and high-scoring results.
final_blend = (0.6 * pred_df['weighted_avg']) + (0.4 * pred_df['rank_ensembled'])

# ===================================================================
# 6. CREATE AND SAVE FINAL SUBMISSION
# ===================================================================
submission_df = pd.DataFrame({'id': pred_df['id'], 'is_cheating': final_blend})
submission_df.to_csv(os.path.join(OUTPUT_DIR, 'submission.csv'), index=False)

print("\nFinal ensembled submission file created successfully!")
print(submission_df.head())

# --- Plotting the results for analysis ---
plt.figure(figsize=(14, 7))

# Plot 1: Distribution of Final Predictions
sns.histplot(submission_df['is_cheating'], bins=50, kde=True, color='green')
plt.title('Distribution of Final Ensembled Predictions on Test Set', fontsize=16)
plt.xlabel('Predicted Probability of Cheating')
plt.ylabel('Frequency')
plt.savefig(os.path.join(PLOT_DIR, "final_prediction_distribution.png"))
plt.show()


# Plot 2: Correlation of Model Predictions
plt.figure(figsize=(8, 6))
correlation_matrix = pred_df[['model_1', 'model_2']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='viridis', fmt=".3f")
plt.title('Correlation Between Model Predictions', fontsize=16)
plt.savefig(os.path.join(PLOT_DIR, "model_correlation_heatmap.png"))
plt.show()

print("\nProcess complete.")